In [28]:
# Read the csv dataset
import py_entitymatching as em
A = em.read_csv_metadata('../datasets/yelp_data.csv')
A['ID'] = range(0, len(A))
em.set_key(A, 'ID')

B = em.read_csv_metadata('../datasets/inspection_data.csv')
B['ID'] = range(0, len(B))
em.set_key(B, 'ID')

print(len(A))
print(len(B))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


16149
20238


In [18]:
A.head()

,name,address,zipcode,cuisine,price,rating,review_count,ID
0,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,$$,4.5,343,0
1,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,$$,4.0,1196,1
2,juniper,"237 w 35th st,",10001,cocktail bars american (new),$$,4.0,180,2
3,haymaker bar and kitchen,"252 w 29th st, ,",10001,gastropubs beer bar,$$,4.0,140,3
4,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,$$,4.0,1232,4


## Blocking

In [ ]:
# Perform blocking
ab = em.AttrEquivalenceBlocker()
C = ab.block_tables(A, B, 'zipcode', 'zipcode', l_output_attrs=['name', 'address', 'zipcode', 'cuisine', 'price'], r_output_attrs=['name', 'address', 'zipcode', 'cuisine', 'violation_code', 'critical_flag', 'grade'])

In [29]:
print(len(C))

2853335


In [19]:
ob = em.OverlapBlocker()
D = ob.block_candset(C, 'address', 'address', overlap_size=4)

0%                          100%
[##############################] | ETA: 00:01:13 | ETA: 00:01:03 | ETA: 00:00:58 | ETA: 00:00:56 | ETA: 00:00:53 | ETA: 00:00:50 | ETA: 00:00:48 | ETA: 00:00:47 | ETA: 00:00:46 | ETA: 00:00:44 | ETA: 00:00:41 | ETA: 00:00:39 | ETA: 00:00:36 | ETA: 00:00:34 | ETA: 00:00:32 | ETA: 00:00:29 | ETA: 00:00:27 | ETA: 00:00:25 | ETA: 00:00:23 | ETA: 00:00:20 | ETA: 00:00:18 | ETA: 00:00:16 | ETA: 00:00:14 | ETA: 00:00:12 | ETA: 00:00:10 | ETA: 00:00:08 | ETA: 00:00:06 | ETA: 00:00:04 | ETA: 00:00:02 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:01:03


In [30]:
# Describe the candidate set
print(len(D))
D.head(2)

1388


,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_zipcode,ltable_cuisine,ltable_price,rtable_name,rtable_address,rtable_zipcode,rtable_cuisine,rtable_violation_code,rtable_critical_flag,rtable_grade
759,759,1,19848,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,$$,dee daa,132 w 31st st,10001,thai,04L,Critical,A
1041,1041,2,14586,juniper,"237 w 35th st,",10001,cocktail bars american (new),$$,juniper,237 w 35th st,10001,american,06C,Critical,A


## Feature extraction

In [21]:
# Adding metadata to the files
em.set_key(A, 'ID')
em.set_key(B, 'ID')

em.set_key(D, '_id')
em.set_fk_ltable(D, 'ltable_ID')
em.set_fk_rtable(D, 'rtable_ID')
em.set_ltable(D, A)
em.set_rtable(D, B)

True

In [22]:
# Manual Feature Extraction
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(A)
atypes2 = em.get_attr_types(B)
match_c = em.get_attr_corres(A, B)
match_f = em.get_features(A, B, atypes1, atypes2, match_c, match_t, match_s)
features_to_drop = list(range(0, 32))
match_f = match_f.drop(features_to_drop)  # We drop all features, as we build features manually.

# Create blackbox features
import re
def custom_address_sim(ltuple, rtuple):
    sim = 0.0
    # Extract all numbers from the address and count how many of them match-
    numbers = re.findall(r'\d+', ltuple['address'])
    num_matches = 0
    for number in numbers:
        if number in rtuple['address']:
            num_matches = num_matches + 1
    if len(numbers) > 0:
        sim = float(num_matches) / float(len(numbers)) # Normalize    
    # Compute jaccard similarity for street names.
    jaccard_sim = em.jaccard(em.tok_alphabetic(ltuple['address']), em.tok_alphabetic(rtuple['address']))
    return sim * jaccard_sim

def custom_name_jaccard(ltuple, rtuple):
    jaccard_sim = em.jaccard(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return jaccard_sim

def custom_address_overlap(ltuple, rtuple):    
    sim = em.overlap_coeff(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim

def custom_name_overlap(ltuple, rtuple):
    overlap_sim = em.overlap_coeff(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return overlap_sim

def custom_address_cosine(ltuple, rtuple):    
    sim = em.cosine(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim

def custom_name_cosine(ltuple, rtuple):
    sim = em.cosine(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return sim

def custom_cuisine_overlap(ltuple, rtuple):
    sim = em.overlap_coeff(em.tok_alphabetic(ltuple['cuisine']), em.tok_alphabetic(rtuple['cuisine']))
    return sim

status = em.add_blackbox_feature(match_f, 'custom_address_sim', custom_address_sim)
status = em.add_blackbox_feature(match_f, 'custom_name_jaccard', custom_name_jaccard)
status = em.add_blackbox_feature(match_f, 'custom_address_overlap', custom_address_overlap)
status = em.add_blackbox_feature(match_f, 'custom_name_cosine', custom_name_cosine)
status = em.add_blackbox_feature(match_f, 'custom_address_cosine', custom_address_cosine)
status = em.add_blackbox_feature(match_f, 'custom_cuisine_overlap', custom_cuisine_overlap)

match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,custom_address_sim,None,None,None,None,None,<function custom_address_sim at 0x128a24840>,None,False
1,custom_name_jaccard,None,None,None,None,None,<function custom_name_jaccard at 0x128a24268>,None,False
2,custom_address_overlap,None,None,None,None,None,<function custom_address_overlap at 0x128a24ae8>,None,False
3,custom_name_cosine,None,None,None,None,None,<function custom_name_cosine at 0x128a24a60>,None,False
4,custom_address_cosine,None,None,None,None,None,<function custom_address_cosine at 0x128a301e0>,None,False
5,custom_cuisine_overlap,None,None,None,None,None,<function custom_cuisine_overlap at 0x128a24950>,None,False


In [23]:
D.head()

,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_zipcode,ltable_cuisine,ltable_price,rtable_name,rtable_address,rtable_zipcode,rtable_cuisine,rtable_violation_code,rtable_critical_flag,rtable_grade
759,759,1,19848,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,$$,dee daa,132 w 31st st,10001,thai,04L,Critical,A
1041,1041,2,14586,juniper,"237 w 35th st,",10001,cocktail bars american (new),$$,juniper,237 w 35th st,10001,american,06C,Critical,A
1643,1643,4,4945,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,$$,mew,53 w 35th st,10001,japanese,06C,Critical,B
3080,3080,8,647,the maze,"32 w 32nd st, fl 3,",10001,korean cocktail bars mexican,$$,the kunjip,32 w 32nd st,10001,korean,02B,Critical,Z
3178,3178,8,4995,the maze,"32 w 32nd st, fl 3,",10001,korean cocktail bars mexican,$$,the maze,32 w 32nd st,10001,korean,06C,Critical,A


In [24]:
H = em.extract_feature_vecs(D, feature_table=match_f, attrs_after=['ltable_name', 'ltable_address', 'ltable_zipcode', 'ltable_cuisine', 'ltable_price', 'rtable_name', 'rtable_address', 'rtable_zipcode', 'rtable_cuisine', 'rtable_violation_code', 'rtable_critical_flag', 'rtable_grade'])
H.head(10)


0%                          100%
[##############################] | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:02


,_id,ltable_ID,rtable_ID,custom_address_sim,custom_name_jaccard,custom_address_overlap,custom_name_cosine,custom_address_cosine,custom_cuisine_overlap,rtable_grade,...,rtable_violation_code,rtable_cuisine,rtable_zipcode,rtable_address,rtable_name,ltable_price,ltable_cuisine,ltable_zipcode,ltable_address,ltable_name
759,759,1,19848,1.00,0.000000,1.0,0.000000,1.000000,0.0,A,...,04L,thai,10001,132 w 31st st,dee daa,$$,american (new) gluten-free breakfast & brunch,10001,"132 w 31st st, ,",friedman's
1041,1041,2,14586,1.00,1.000000,1.0,1.000000,1.000000,1.0,A,...,06C,american,10001,237 w 35th st,juniper,$$,cocktail bars american (new),10001,"237 w 35th st,",juniper
1643,1643,4,4945,0.75,0.500000,1.0,0.707107,0.894427,0.0,B,...,06C,japanese,10001,53 w 35th st,mew,$$,sushi bars izakaya cocktail bars,10001,"53 w 35th st, basement,",izakaya mew
3080,3080,8,647,0.75,0.333333,1.0,0.500000,0.816497,1.0,Z,...,02B,korean,10001,32 w 32nd st,the kunjip,$$,korean cocktail bars mexican,10001,"32 w 32nd st, fl 3,",the maze
3178,3178,8,4995,0.75,1.000000,1.0,1.000000,0.816497,1.0,A,...,06C,korean,10001,32 w 32nd st,the maze,$$,korean cocktail bars mexican,10001,"32 w 32nd st, fl 3,",the maze
3675,3675,9,12214,1.00,0.000000,1.0,0.000000,1.000000,0.0,A,...,10I,café/coffee/tea,10001,12 w 32nd st,gong cha,$,korean,10001,"12 w 32nd st, ,",woorijip authentic korean food
3960,3960,10,6195,1.00,1.000000,1.0,1.000000,1.000000,1.0,B,...,06B,korean,10001,22 w 32nd st,jongro bbq,$$,korean barbeque beer bar,10001,"22 w 32nd st, ,",jongro bbq
4251,4251,11,1588,1.00,0.500000,1.0,0.707107,1.000000,1.0,A,...,10F,korean,10001,9 w 32nd st,5 senses,$$,korean barbeque,10001,"9 w 32nd st, ,",five senses
5429,5429,14,2712,1.00,1.000000,1.0,1.000000,1.000000,1.0,A,...,04L,american,10001,140 w 30th st,till & sprocket,$$,american (new) cafes bars,10001,"140 w 30th st, ,",till & sprocket
6974,6974,18,3747,1.00,1.000000,1.0,1.000000,1.000000,1.0,B,...,08A,french,10001,245 w 29th st,ocabanon,$$,french wine bars tapas bars,10001,"245 w 29th st,",ocabanon


## Matching

In [27]:
from sklearn.externals import joblib
matcher = joblib.load('../../stage-3/code/naive_bayes_trained.pkl')
# Predict on H
predictions = matcher.predict(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID','ltable_name', 'ltable_address', 'ltable_zipcode', 'ltable_cuisine', 'ltable_price', 'rtable_name', 'rtable_address', 'rtable_zipcode', 'rtable_cuisine', 'rtable_violation_code', 'rtable_critical_flag', 'rtable_grade'], 
              append=True, target_attr='predicted', inplace=False)
em.to_csv_metadata(predictions, '../datasets/matched_A_B.csv')

True